# Expense Claim Analysis

This notebook demonstrates how to create agents to generate an expense claim email and visualize expense data using a pie chart. The following steps explain each part of the notebook.

1. Create an agent to generate an expense claim email.
2. Create an agent to visualize expense data using a pie chart.

Created by:

<table>
<tr>
    <td align="center"><a href="https://github.com/ShivamGoyal03">
        <img src="https://github.com/ShivamGoyal03.png" width="100px;" alt="Shivam Goyal"/><br />
        <sub><b>Shivam Goyal</b></sub>
    </a><br />
    </td>
</tr></table>

Example of a travel expense scenario:
Imagine you're an employee traveling for a business meeting in another city. Your company has a policy to reimburse all reasonable travel-related expenses. Here’s a breakdown of potential travel expenses:
- Transportation:
Airfare for a round trip from your home city to the destination city.
Taxi or ride-hailing services to and from the airport.
Local transportation in the destination city (like public transit, rental cars, or taxis).

- Accommodation:
Hotel stay for three nights at a mid-range business hotel close to the meeting venue.

- Meals:
Daily meal allowance for breakfast, lunch, and dinner, based on the company's per diem policy.

- Miscellaneous Expenses:
Parking fees at the airport.
Internet access charges at the hotel.
Tips or small service charges.

- Documentation:
You submit all receipts (flights, taxis, hotel, meals, etc.) and a completed expense report for reimbursement.

## Install necessary packages

First, install the required Python packages: `pandas` and `plotly`.

In [1]:
%pip install pandas plotly

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import required libraries

Import the necessary libraries and modules for the notebook.

In [10]:
import os
from dotenv import load_dotenv
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import ChatHistoryTruncationReducer
from semantic_kernel.functions import kernel_function
import plotly.express as px
import pandas as pd
from pydantic import BaseModel, Field
from typing import List

 ## Define Expense Models

 Create a Pydantic model for individual expenses and an ExpenseFormatter class to convert a user query into structured expense data.

 Each expense will be represented in the format:
 `{'date': '07-Mar-2025', 'description': 'flight to destination', 'amount': 675.99, 'category': 'Transportation'}`


In [11]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query and converts it into a list of Expense objects.
        Expected format for each expense: "date|description|amount|category"
        Expenses should be separated by semicolons.
        
        Example:
        "07-Mar-2025|flight to destination|675.99|Transportation; 07-Mar-2025|taxi from airport to hotel|24.00|Transportation"
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    expense = Expense(
                        date=date.strip(),
                        description=description.strip(),
                        amount=float(amount.strip()),
                        category=category.strip()
                    )
                    expense_list.append(expense)
        return expense_list

## Defining Agents - Generating the Email

Create an agent class to generate an email for submitting an expense claim.
- This agent uses the `kernel_function` decorator to define a function that generates an email for submitting an expense claim.
- It calculates the total amount of the expenses and formats the details into an email body.

In [12]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

### Agent for Generating the Chart

Create an agent class to generate a pie chart using Plotly to visualize the distribution of expenses by category.
- This agent uses the `kernel_function` decorator to define a function that creates a pie chart using Plotly.
- It converts the expenses data into a Pandas DataFrame and generates a pie chart showing the distribution of expenses by category.


In [13]:
class ExpenseChartAgent:
    
    @kernel_function(description="Create a pie chart to visualize the distribution of expenses by category")
    def create_expense_chart(expenses):
        df = pd.DataFrame(expenses)
        fig = px.pie(df, values='amount', names='category', title='Expense Distribution by Category')
        return fig

## Processing Expenses

Define an asynchronous function to process the expenses by creating and registering the necessary agents and then invoking them.
- This function processes the expenses by loading environment variables, creating the necessary agents, and registering them as plugins.
- It creates a group chat with the two agents and sends a prompt message to generate the email and pie chart based on the expenses data.
- It handles any errors that occur during the chat invocation and ensures proper cleanup of the agents.

In [14]:
async def process_expenses():
    # Load environment variables from .env file
    load_dotenv()
    ai_agent_settings = AzureAIAgentSettings.create()
    kernel = Kernel()

    # Create expense claim data
    user_query = (
        "07-Mar-2025|flight to destination|675.99|Transportation; "
        "07-Mar-2025|taxi from airport to hotel|24.00|Transportation; "
        "07-Mar-2025|dinner|65.50|Meals; "
        "07-Mar-2025|hotel stay|125.90|Accommodation; "
        "07-Mar-2025|parking at airport|20.00|Miscellaneous; "
        "07-Mar-2025|internet at hotel|10.00|Miscellaneous; "
        "08-Mar-2025|breakfast|12.00|Meals; "
        "08-Mar-2025|lunch|14.70|Meals; "
        "08-Mar-2025|dinner|45.50|Meals; "
        "08-Mar-2025|hotel stay|125.90|Accommodation; "
        "08-Mar-2025|internet at hotel|10.00|Miscellaneous; "
        "08-Mar-2025|tip for dinner|9.00|Miscellaneous; "
        "09-Mar-2025|breakfast|10.00|Meals; "
        "09-Mar-2025|taxi to airport|25.90|Transportation; "
        "09-Mar-2025|flight back|675.99|Transportation; "
        "09-Mar-2025|parking at airport|20.00|Miscellaneous"
    )

    # Parse the user query to extract expenses
    formatter = ExpenseFormatter(raw_query=user_query)
    expenses = [expense.model_dump() for expense in formatter.parse_expenses()]

    # Connect to the Azure AI Foundry project
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(
            credential=creds,
        ) as project_client,
    ):

        # Define and register the email agent
        email_agent_def = await project_client.agents.create_agent(
            model=ai_agent_settings.model_deployment_name,
            name="email_agent",
            instructions="Generate a professional expense claim email with formatted details."
        )
        email_agent = AzureAIAgent(
            client=project_client,
            definition=email_agent_def
        )

        # Define and register the chart agent
        chart_agent_def = await project_client.agents.create_agent(
            model=ai_agent_settings.model_deployment_name,
            name="chart_agent",
            instructions="Create a chart showing expense distribution by category."
        )
        chart_agent = AzureAIAgent(
            client=project_client,
            definition=chart_agent_def
        )

        # Register the agents as plugins
        kernel.add_plugin(ExpenseEmailAgent, plugin_name="expenseEmailAgent")
        kernel.add_plugin(ExpenseChartAgent, plugin_name="expenseChartAgent")

        # Create a group chat with the two agents
        history_reducer = ChatHistoryTruncationReducer(target_count=3)
        chat = AgentGroupChat(
            agents=[email_agent, chart_agent],
            selection_strategy=SequentialSelectionStrategy(initial_agent=email_agent, history_reducer=history_reducer),
            termination_strategy=DefaultTerminationStrategy(maximum_iterations=2, history_reducer=history_reducer)
        )

        try:
            # Add the user input as a chat message
            prompt_message = "Generate an expense claim email and create a chart based on the following expenses: " + str(expenses)
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=prompt_message)
            )
            print(f"# AuthorRole.USER: \n{prompt_message}\n")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: \n{content.content}\n")

        except Exception as e:
            print(f"Error during chat invocation: {e}")
        finally:
            await chat.reset()
            await project_client.agents.delete_agent(email_agent.id)
            await project_client.agents.delete_agent(chart_agent.id)

## Main function

Define the main function to clear the console and run the `process_expenses` function asynchronously.

In [15]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# AuthorRole.USER: 
Generate an expense claim email and create a chart based on the following expenses: [{'date': '07-Mar-2025', 'description': 'flight to destination', 'amount': 675.99, 'category': 'Transportation'}, {'date': '07-Mar-2025', 'description': 'taxi from airport to hotel', 'amount': 24.0, 'category': 'Transportation'}, {'date': '07-Mar-2025', 'description': 'dinner', 'amount': 65.5, 'category': 'Meals'}, {'date': '07-Mar-2025', 'description': 'hotel stay', 'amount': 125.9, 'category': 'Accommodation'}, {'date': '07-Mar-2025', 'description': 'parking at airport', 'amount': 20.0, 'category': 'Miscellaneous'}, {'date': '07-Mar-2025', 'description': 'internet at hotel', 'amount': 10.0, 'category': 'Miscellaneous'}, {'date': '08-Mar-2025', 'description': 'breakfast', 'amount': 12.0, 'category': 'Meals'}, {'date': '08-Mar-2025', 'description': 'lunch', 'amount': 14.7, 'category': 'Meals'}, {'date': '08-Mar-2025', 'description': 'dinner', 'amount': 45.5, 'category': 'Meals'}, {'d